<h2>C - Network

In [1]:
import pandas as pd
import icecream as ic
import numpy as np
import plotly
import matplotlib as plt
import xml.etree.ElementTree as ET
from inspect import getmembers, isclass, isfunction1007
import pandas as pd

small-molecule roles

C1: small-molecule roles

C2: metabolic pathways

C3: signaling pathways

C4: biological processes

C5: interactomes